In [0]:
# Install necessary libraries
#%pip install azure-eventhub
#%pip install azure-storage-file-datalake

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
  Using cached azure_eventhub-5.13.0-py3-none-any.whl (325 kB)
  Using cached azure_core-1.32.0-py3-none-any.whl (198 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.4.0
    Not uninstalling typing-extensions at /databricks/python3/lib/python3.10/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-a8c6a8ca-5ac0-4c07-8833-c77e6b231215
    Can't uninstall 'typing_extensions'. No files were found to uninstall.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.3/258.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.4/408.

In [0]:
%pip install azure-identity

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.6/187.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.2/113.2 kB 4.8 MB/s eta 0:00:00
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.library.restartPython()

In [0]:
from azure.eventhub import EventHubConsumerClient
from azure.storage.filedatalake import DataLakeServiceClient
from azure.identity import ClientSecretCredential
import json
import os

In [0]:
dbutils.secrets.list(scope="my_scope")

[SecretMetadata(key='datalakesecret')]

In [0]:


# Service Principal credentials
tenant_id = "254beea4-dccf-4626-8762-246272629bd5"
client_id = "4ea70484-a0ed-4293-932e-0cb82d189cf9"
client_secret = dbutils.secrets.get(scope="my_scope", key="datalakesecret")

# Authenticate with Azure
credential = ClientSecretCredential(tenant_id, client_id, client_secret)

# Initialize DataLakeServiceClient
account_url = f"https://weatherdlake.dfs.core.windows.net"
service_client = DataLakeServiceClient(account_url=account_url, credential=credential)


In [0]:
# Azure Event Hubs connection details
event_hub_connection_string = "Endpoint=sb://weathereventdata.servicebus.windows.net/;SharedAccessKeyName=weatherdatahubpolicy;SharedAccessKey=iDYXdrCDuhitLRbc02/VWgqsl0lt41KGy+AEhIS/Oj4=;EntityPath=weatherdatahub"
event_hub_name = "weatherdatahub"
consumer_group = "$Default"

In [0]:
# Setup the Event Hub Consumer Client
def on_event(partition_context, event):
    try:
        # Deserialize the event data
        event_data = json.loads(event.body_as_str())
        city_name = event_data.get("name", "Unknown_City")
        print(f"Captured event for city: {city_name}")
        
        store_data_in_datalake(city_name, event_data)
        
    except Exception as e:
        print(f"Error processing event: {e}")

In [0]:
# Function to store data in Azure Data Lake
def store_data_in_datalake(city_name, data):
    try:
        filesystem_client = service_client.get_file_system_client("eventhubdata")
        directory_client = filesystem_client.get_directory_client("weatherdata")
        
        file_name = f"{city_name}_weather_data.json"
        
        file_client = directory_client.create_file(file_name)
        
        data_json = json.dumps(data)
        file_client.append_data(data_json, offset=0, length=len(data_json))
        file_client.flush_data(len(data_json))
        
        print(f"Data for {city_name} stored in Data Lake successfully!")
    
    except Exception as e:
        print(f"Error storing data in Data Lake: {e}")



In [0]:
client = EventHubConsumerClient.from_connection_string(event_hub_connection_string, consumer_group, eventhub_name=event_hub_name)

In [0]:
# Start receiving data from Event Hubs
try:
    with client:
        print("Starting to receive events from Event Hubs...")
        client.receive(
            on_event=on_event,
            starting_position="-1",
            partition_id="0"    
        )
except KeyboardInterrupt:
    print("Receiving stopped.")
finally:
    client.close()

Starting to receive events from Event Hubs...
Captured event for city: Hyderabad
Data for Hyderabad stored in Data Lake successfully!
Captured event for city: Mumbai
Data for Mumbai stored in Data Lake successfully!
Captured event for city: Delhi
Data for Delhi stored in Data Lake successfully!
Captured event for city: Chennai
Data for Chennai stored in Data Lake successfully!
Captured event for city: Bengaluru
Data for Bengaluru stored in Data Lake successfully!
Captured event for city: Kolkata
Data for Kolkata stored in Data Lake successfully!
Captured event for city: Pune
Data for Pune stored in Data Lake successfully!
Captured event for city: Jaipur
Data for Jaipur stored in Data Lake successfully!
Captured event for city: Ahmedabad
Data for Ahmedabad stored in Data Lake successfully!
Captured event for city: Surat
Data for Surat stored in Data Lake successfully!
Captured event for city: Lucknow
Data for Lucknow stored in Data Lake successfully!
Captured event for city: Kanpur
Data

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:136)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:730)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:448)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:448)
	at com.databricks.spark.chauffeur.ChauffeurState.cancelExecutio